# Predicting Product Condition: New vs Used

----------------------------------------------------
by Natalia López Gallego

This notebook performs trains a model to predict whether a product is new or used based on various seller and location features.

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from xgboost import XGBClassifier


In [4]:
# Load dataset
df = pd.read_csv('df_cleaned.csv')
df.head()

,condition,status,price,buying_mode,available_quantity,sold_quantity,accepts_mercadopago,automatic_relist,shipping_mode,address_state_name,tags,sub_status,title_length,address_city_name_freq,year_created,month_created,days_since_created,listing_duration,days_since_last_updated
0,1,active,80.0,buy_it_now,1,0,True,False,not_specified,Capital Federal,dragged_bids_and_visits,NaN,60,202,2015,9,3534,60,3534
1,0,active,2650.0,buy_it_now,1,0,True,False,me2,Capital Federal,NaN,NaN,57,3420,2015,9,3513,60,3513
2,0,active,60.0,buy_it_now,1,0,True,False,me2,Capital Federal,dragged_bids_and_visits,NaN,43,652,2015,9,3529,60,3529
3,1,active,580.0,buy_it_now,1,0,True,False,me2,Capital Federal,NaN,NaN,52,824,2015,9,3511,66,3504
4,0,active,30.0,buy_it_now,1,0,True,False,not_specified,Buenos Aires,dragged_bids_and_visits,NaN,25,230,2015,8,3546,60,3542


In [5]:
# Overview of the data
df.info()
df.describe(include='object')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   condition                100000 non-null  int64  
 1   status                   100000 non-null  object 
 2   price                    100000 non-null  float64
 3   buying_mode              100000 non-null  object 
 4   available_quantity       100000 non-null  int64  
 5   sold_quantity            100000 non-null  int64  
 6   accepts_mercadopago      100000 non-null  bool   
 7   automatic_relist         100000 non-null  bool   
 8   shipping_mode            100000 non-null  object 
 9   address_state_name       100000 non-null  object 
 10  tags                     75090 non-null   object 
 11  sub_status               986 non-null     object 
 12  title_length             100000 non-null  int64  
 13  address_city_name_freq   100000 non-null  int64  
 14  year_

,status,buying_mode,shipping_mode,address_state_name,tags,sub_status
count,100000,100000,100000,100000,75090,986
unique,4,3,4,24,7,3
top,active,buy_it_now,me2,Capital Federal,dragged_bids_and_visits,suspended
freq,95675,97001,51255,57857,72551,966


In [9]:
# 1. Splitting the data into training and testing sets
X = df.drop(columns=['condition']) 
y = df['condition']  # Replace with your target column

X_encoded = pd.get_dummies(X, drop_first=True)  # Convert all string columns

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)


In [ ]:

# 2. Train multiple classification models

models = {
    'Logistic Regression': LogisticRegression(),
    'Random Forest': RandomForestClassifier(),
    'SVM': SVC(probability=True), 
    'XGBoost': XGBClassifier()
}

results = {}

for model_name, model in models.items():
  
    model.fit(X_train, y_train)
    
    # Predictions
    y_pred = model.predict(X_test)
    y_pred_prob = model.predict_proba(X_test)[:, 1] 
    # Metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1_score = classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score']
    roc_auc = roc_auc_score(y_test, y_pred_prob) if len(y_pred_prob.shape) == 1 else 'N/A'  
    # Results
    results[model_name] = {
        'Accuracy': accuracy,
        'F1-Score': f1_score,
        'ROC-AUC': roc_auc
    }

c:\Users\ntlg2\Documents\ml_exercise\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# 3. Printing the results
results_df = pd.DataFrame(results).T
print(results_df)

In [ ]:
# 4. Saving the  DataFrames
# Saving the training and testing data, and predictions
# Crear un DataFrame de los resultados

X_train.to_csv('X_train.csv', index=False)
X_test.to_csv('X_test.csv', index=False)
y_test.to_csv('y_test.csv', index=False)